In [103]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [119]:
COLUMNS_TO_DROP = [
    "name",
    "description",
    "listing_url",
    "scrape_id",
    "last_scraped",
    "source",
    "picture_url",
    "host_name",
    "host_about",
    "host_url",
    "host_verifications",
    "host_location",
    "neighborhood_overview",
    "neighbourhood",
    "host_neighbourhood", #neighbourhood_cleansed used instead
    "neighbourhood_group_cleansed",
    "host_thumbnail_url",
    "host_picture_url",
    "property_type", #use room_type instead as it isn't freeform text
    "license",
    "calendar_updated",
    "calendar_last_scraped"
]

COLUMNS_TO_RENAME = {'price': 'price_nok'}
DATE_COLUMNS = ["host_since","first_review","last_review"]


In [120]:
df_listings = pd.read_csv("listings.csv")
df_listings.head(5)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,42932,https://www.airbnb.com/rooms/42932,20240629155744,2024-06-29,city scrape,"Charming apartment, Oslo Center, Ro",NaN,(URL HIDDEN),https://a0.muscache.com/pictures/miso/Hosting-...,187463,...,4.96,4.96,4.85,NaN,f,2,2,0,0,0.39
1,71725,https://www.airbnb.com/rooms/71725,20240629155744,2024-06-29,city scrape,Central big room - ap with balcony,A bargain in Oslo!!<br />Room in new and fully...,The apartment is situated in the eastern part ...,https://a0.muscache.com/pictures/468893/9c8190...,368229,...,4.94,4.71,4.80,NaN,f,1,0,1,0,0.34
2,85902,https://www.airbnb.com/rooms/85902,20240629155744,2024-06-29,city scrape,"Stylish design Apt, super central",Discount on longer stays - Super central locat...,A very central location on the east side of to...,https://a0.muscache.com/pictures/fcb4770a-e06f...,250159,...,4.83,4.49,4.71,NaN,f,2,1,1,0,0.50
3,123041,https://www.airbnb.com/rooms/123041,20240629155744,2024-06-29,city scrape,"Cozy room in design apartment, central east side",Welcome to a relaxing stay at a creative desig...,A very central location on the east side of to...,https://a0.muscache.com/pictures/fc30957d-cbbe...,250159,...,4.85,4.61,4.76,NaN,f,2,1,1,0,1.48
4,149776,https://www.airbnb.com/rooms/149776,20240629155744,2024-06-30,city scrape,Bird's nest in the heart of Oslo,"A double room in a spacious, elegant apartment...","Despite its central location, the neighbourhoo...",https://a0.muscache.com/pictures/fc1eb1a5-0ab8...,714768,...,4.95,4.88,4.83,NaN,f,2,0,2,0,0.27


In [121]:
# Drop columns that are defined as generally unnecessary for our problem and rename where necessary
def drop_columns_by_list (df, columns_to_delete):
    column_filter = df.filter(columns_to_delete)
    df.drop(column_filter, inplace=True, axis=1)
    return df

df_listings = drop_columns_by_list(df=df_listings,columns_to_delete=COLUMNS_TO_DROP)
df_listings.rename(columns=COLUMNS_TO_RENAME,inplace=True)

# Convert prive column
if df_listings['price_nok'].dtype != "float":
    # Remove the dollar sign and commas, then convert to float column
    df_listings['price_nok'] = df_listings['price_nok'].str.replace('$', '').str.replace(',', '').astype(float)

df_listings["price_nok"].head(5)

C:\Users\lukas\AppData\Local\Temp\ipykernel_21516\524871180.py:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_listings['price_nok'] = df_listings['price_nok'].str.replace('$', '').str.replace(',', '').astype(float)


0    3400.0
1     572.0
2    1929.0
3     736.0
4    1029.0
Name: price_nok, dtype: float64

In [122]:
# Helper function to check if a value is 't', 'f', or NaN
def is_tf_nan(value):
    return value in ['t', 'f'] or pd.isna(value)

# Convert boolean (t,f) to 0,1
def find_and_convert_boolean_columns(df):
    # Identify columns that contain only 't', 'f', or NaN
    columns_to_convert = df.columns[df.applymap(is_tf_nan).all()]
    print("Boolean columns to be converted:", columns_to_convert)
    df[columns_to_convert] = df[columns_to_convert].replace({'t': 1, 'f': 0}).fillna(0).astype(int)
    return df

# Remove percentage signs and convert to float
def find_and_convert_percentage_columns(df):
    percentage_columns = []

    # Identify columns with percentage values
    for column in df.columns:
        if df[column].dropna().astype(str).str.endswith('%').any():
            percentage_columns.append(column)
    
    print("Percentage columns to be converted:", percentage_columns)
    # Convert identified columns to decimal format
    for column in percentage_columns:
        df[column] = df[column].str.replace('%', '').astype(float) / 100
        df[column] = df[column].fillna(0)  # Handle NaN values if needed

    return df

def find_and_convert_date_columns(df,date_columns_to_convert,date_format='%Y-%m-%d'):
    column_filter = df.filter(date_columns_to_convert)
    
    print("Date columns to be converted:", column_filter.columns)
    # Convert identified columns to datetime
    for column in column_filter:
        df[column] = pd.to_datetime(df[column], format=date_format)
    
    return df


df_listings = find_and_convert_boolean_columns(df_listings)
df_listings = find_and_convert_percentage_columns(df_listings)
df_listings = find_and_convert_date_columns(df_listings, date_columns_to_convert=DATE_COLUMNS)
df_listings.head(5)

Boolean columns to be converted: Index(['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
       'has_availability', 'instant_bookable'],
      dtype='object')
Percentage columns to be converted: ['host_response_rate', 'host_acceptance_rate']
Date columns to be converted: Index(['host_since', 'first_review', 'last_review'], dtype='object')


,id,host_id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_listings_count,host_total_listings_count,host_has_profile_pic,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,42932,187463,2010-08-01,within an hour,1.0,1.00,1,2,2,1,...,4.94,4.96,4.96,4.85,0,2,2,0,0,0.39
1,71725,368229,2011-01-30,within a few hours,1.0,0.67,0,1,1,1,...,4.98,4.94,4.71,4.80,0,1,0,1,0,0.34
2,85902,250159,2010-10-01,within an hour,1.0,0.96,1,3,3,1,...,4.73,4.83,4.49,4.71,0,2,1,1,0,0.50
3,123041,250159,2010-10-01,within an hour,1.0,0.96,1,3,3,1,...,4.85,4.85,4.61,4.76,0,2,1,1,0,1.48
4,149776,714768,2011-06-18,within an hour,1.0,0.67,0,2,2,1,...,4.98,4.95,4.88,4.83,0,2,0,2,0,0.27


In [118]:
# Inspect columns that contain Nan values and decide how to handle them
df_listings.isna().sum().sort_values(ascending=False)

review_scores_value                             2739
review_scores_location                          2738
review_scores_accuracy                          2738
review_scores_cleanliness                       2738
review_scores_checkin                           2738
review_scores_communication                     2738
reviews_per_month                               2714
first_review                                    2714
review_scores_rating                            2714
last_review                                     2714
host_response_time                              2117
price_nok                                       1249
bathrooms                                       1215
beds                                            1210
bedrooms                                         173
bathrooms_text                                     6
availability_30                                    0
availability_60                                    0
availability_90                               